## Importing Necessary Packages

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split
import cv2
import random
from tensorflow.keras import layers
from tensorflow.keras import activations
import datetime
from tensorboard.plugins.hparams import api as hp

## Defining Data directory

In [2]:
data_dir = os.getcwd() + '\\RTAI\\dataset\\'
Classes = ["Alpaca", "Not Alpaca"]
image_size = 28
X = list()
y = list()

In [3]:
def create_dataset(data_dir):
    train_data = []
    X = []
    y = []
    for each in Classes:
        Class_id = Classes.index(each)
        path = os.path.join(data_dir, each)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            converted_array = cv2.resize(img_array, (image_size, image_size))
            train_data.append([converted_array, Class_id])
    
    random.shuffle(train_data)
    
    for feature, label in train_data:
        X.append(feature)
        y.append(label)
    X = np.array(X)
    y = np.array(y)
    return X, y

In [4]:
X, y = create_dataset(data_dir)
X_dup = np.array(X)
assert(X.shape[0] == y.shape[0])

In [5]:
X = np.array(X).reshape(-1 , 1, image_size, image_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [6]:
X_train, X_test, X_val = tf.cast(X_train, tf.float32), tf.cast(X_test,tf.float32), tf.cast(X_val,tf.float32)

In [7]:
len(X_train)
print(X_train.shape)

(228, 1, 28, 28)


In [8]:
len(X_test)

49

In [9]:
len(X_val)

50

## Custom Layer and Model Creation

### Custom Layer

In [10]:
class Proposed_Layer(tf.keras.layers.Layer):
    def __init__(self, filters):
        super(Proposed_Layer, self).__init__()
        self.filters = filters
        self.bn = tf.keras.layers.BatchNormalization()

    def build(self, input_shape):
        self.weight = self.add_weight(
            shape=(self.filters, input_shape[-1], input_shape[-1]),
            initializer="random_normal",
            trainable=True,
        )

    def call(self, inputs):
        x = self.bn(inputs)
        samples, fmaps_stored = tf.shape(x)[0], tf.shape(x)[1]
        resultant = tf.zeros((samples, self.filters, tf.shape(x)[2]-2, tf.shape(x)[3]-2))
        resultant_shape = tf.shape(resultant)
        dim_sample, dim_channel, dim_row, dim_col = resultant_shape[0], resultant_shape[1], resultant_shape[2], resultant_shape[3]

        for n in range(dim_sample):
            for k in range(dim_channel):
                for i in range(dim_row):
                    for j in range(dim_col):
                        total_weight = tf.constant(0.0)
                        for each_param in range(fmaps_stored):
                            x1, y1, x2, y2 = i, i+3, j, j+3
                            kernel = x[n, each_param, x1:y1, x2:y2]
                            weights = self.weight[k, x1:y1, x2:y2]
                            total_weight += tf.math.reduce_sum(tf.matmul(kernel, weights))
                        resultant = tf.tensor_scatter_nd_add(resultant, [[n, k, i, j]], [total_weight])

        return tf.reshape(resultant, (dim_sample, dim_channel, dim_row, dim_col))

## Model Architecture

In [11]:
def create_model():
    # Defining the Sequential model
    model = tf.keras.models.Sequential()

    # Adding a custom layer with 16 filters
    model.add(Proposed_Layer(filters= 5))

    # Adding a ReLU activation layer
    model.add(tf.keras.layers.Activation(activations.relu))

    # Adding a MaxPooling2D layer to downsample the feature maps
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2), strides=(2, 2),
    ))

    # Adding another custom layer with 12 filters
    model.add(Proposed_Layer(filters=3))

    # Adding a ReLU activation layer
    model.add(tf.keras.layers.Activation(activations.relu))

    # Adding another MaxPooling2D layer to downsample the feature maps
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2), strides=(2, 2),
    ))

    # Adding another custom layer with 8 filters and no activation function
    # This layer generates 8 feature maps
    model.add(Proposed_Layer(filters=2))

    # Adding a ReLU activation layer
    model.add(layers.Activation(activations.relu))

    # Adding a Flatten layer to convert the 2D feature maps into a 1D array
    model.add(layers.Flatten())

    # Adding a fully connected Dense layer with 16 neurons and a ReLU activation
    model.add(layers.Dense(units=16, activation="relu"))

    # Adding an output layer with one neuron and a softmax activation
    # This will predict the class probability for binary classification
    model.add(layers.Dense(units=1, activation="softmax"))

    return model

In [12]:
def model_builder(model_custom, alpha, epoch, batch_size = None, adam=False):
    # Set optimizer based on whether adam is used or not
    if adam:
        optimizer = tf.keras.optimizers.Adam(learning_rate=alpha)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=alpha)

    # Compile the model using the chosen optimizer
    model_custom.compile(optimizer=optimizer, loss='BinaryCrossentropy', metrics=['accuracy'])

    # Set up directory to store run files for TensorBoard
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Train the model and save the training history
    with tf.summary.create_file_writer(log_dir).as_default():
        model_history = model_custom.fit(X_train, y_train,
                                     epochs=epoch,
                                     batch_size=batch_size,
                                     validation_data=(X_val, y_val),
                                     callbacks=[tensorboard_callback])

    # Calculate and save the training accuracy
    train_acc = np.mean(model_history.history['accuracy'])
    
    # Evaluate the model's performance on the test data
    test_loss, test_acc = model_custom.evaluate(X_test, y_test)
    train_loss = model_history.history['loss']

    # Save results in a dictionary
    results = {"Learning Rate": alpha, "Batch Size": batch_size, "Epoch": epoch,
                  "Train Accuracy": train_acc, "Test Accuracy": test_acc, "Train Loss": train_loss, "Test Loss": test_acc}

    return model_history, results


In [13]:
model_custom = create_model()

In [ ]:
model_history1, results1 = model_builder(model_custom, alpha = 0.05, epoch = 5, batch_size = 32, adam=False)

# RUN ONLY UNTIL ABOVE CELL

In [14]:
model_history2, results2 = model_builder(model_custom, alpha = 0.05, epoch = 6, batch_size = 64, adam=True)

Epoch 1/6
4/4 [==============================] - 415s 101s/step - loss: 0.7841 - accuracy: 0.5965 - val_loss: 79.3690 - val_accuracy: 0.4400
Epoch 2/6
4/4 [==============================] - 407s 100s/step - loss: 0.5967 - accuracy: 0.5965 - val_loss: 63.5287 - val_accuracy: 0.4400
Epoch 3/6
4/4 [==============================] - 414s 103s/step - loss: 0.5424 - accuracy: 0.5965 - val_loss: 32.8001 - val_accuracy: 0.4400
Epoch 4/6
4/4 [==============================] - 387s 96s/step - loss: 0.5333 - accuracy: 0.5965 - val_loss: 33.5009 - val_accuracy: 0.4400
Epoch 5/6
4/4 [==============================] - 385s 95s/step - loss: 0.5169 - accuracy: 0.5965 - val_loss: 37.8529 - val_accuracy: 0.4400
Epoch 6/6
4/4 [==============================] - 386s 95s/step - loss: 0.4761 - accuracy: 0.5965 - val_loss: 35.8348 - val_accuracy: 0.4400


NameError: name 'model_nn' is not defined

In [ ]:
model_history3, results3 = model_builder(model_custom, alpha = 0.1, epoch = 5, batch_size = 32, adam=False)

In [ ]:
model_history4, results4 = model_builder(model_custom, alpha = 0.1, epoch = 6, batch_size = 64, adam=True)

## Reducing Model depth

#### Re-writing the create model function

In [ ]:
def create_model():
    # Defining the Sequential model
    model = tf.keras.models.Sequential()

    # Adding another custom layer with 12 filters
    model.add(Proposed_Layer(filters=4))

    # Adding a ReLU activation layer
    model.add(tf.keras.layers.Activation(activations.relu))

    # Adding another MaxPooling2D layer to downsample the feature maps
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2), strides=(2, 2),
    ))

    # Adding another custom layer with 8 filters
    # This layer generates 8 feature maps
    model.add(Proposed_Layer(filters=2))

    # Adding a ReLU activation layer
    model.add(layers.Activation(activations.relu))

    # Adding a Flatten layer to convert the 2D feature maps into a 1D array
    model.add(layers.Flatten())

    # Adding a fully connected Dense layer with 16 neurons and a ReLU activation
    model.add(layers.Dense(units=16, activation="relu"))

    # Adding an output layer with one neuron and a softmax activation
    # This will predict the class probability for binary classification
    model.add(layers.Dense(units=1, activation="softmax"))

    return model

In [ ]:
model_depth = create_model()
model_historyd, resultsd = model_builder(model_depth, alpha = 0.1, epoch = 5, batch_size = 32, adam=False)

## Decreasing Model Width

#### Re-writing the create model function

In [ ]:
def create_model():
    # Defining the Sequential model
    model = tf.keras.models.Sequential()

    # Adding a custom layer with 8 filters
    model.add(Proposed_Layer(filters= 4))

    # Adding a ReLU activation layer
    model.add(tf.keras.layers.Activation(activations.relu))

    # Adding a MaxPooling2D layer to downsample the feature maps
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2), strides=(2, 2),
    ))

    # Adding another custom layer with 4 filters
    model.add(Proposed_Layer(filters=3))

    # Adding a ReLU activation layer
    model.add(tf.keras.layers.Activation(activations.relu))

    # Adding another MaxPooling2D layer to downsample the feature maps
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2), strides=(2, 2),
    ))

    # Adding another custom layer with 2 filters and no activation function
    # This layer generates 2 feature maps
    model.add(Proposed_Layer(filters=2))

    # Adding a ReLU activation layer
    model.add(layers.Activation(activations.relu))

    # Adding a Flatten layer to convert the 2D feature maps into a 1D array
    model.add(layers.Flatten())

    # Adding a fully connected Dense layer with 16 neurons and a ReLU activation
    model.add(layers.Dense(units=16, activation="relu"))

    # Adding an output layer with one neuron and a softmax activation
    # This will predict the class probability for binary classification
    model.add(layers.Dense(units=1, activation="softmax"))

    return model

In [ ]:
model_width = create_model()
model_historyw, resultsw = model_builder(model_width, alpha = 0.1, epoch = 5, batch_size = 32, adam=False)

## Default CNN

In [ ]:
def create_model():
    # Defining the Sequential model
    model = tf.keras.models.Sequential()

    # Adding a Convolution layer with 16 filters
    model.add(Conv2D(filters= 4, kerne_size=(3,3)))

    # Adding a ReLU activation layer
    model.add(tf.keras.layers.Activation(activations.relu))

    # Adding a MaxPooling2D layer to downsample the feature maps
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2), strides=(2, 2),
    ))

    # Adding a Convolution layer with 16 filters
    model.add(Conv2D(filters= 3, kerne_size=(3,3)))

    # Adding a ReLU activation layer
    model.add(tf.keras.layers.Activation(activations.relu))

    # Adding another MaxPooling2D layer to downsample the feature maps
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2), strides=(2, 2),
    ))

    # Adding a Convolution layer with 16 filters
    model.add(Conv2D(filters= 2, kerne_size=(3,3)))

    # Adding a ReLU activation layer
    model.add(layers.Activation(activations.relu))

    # Adding a Flatten layer to convert the 2D feature maps into a 1D array
    model.add(layers.Flatten())

    # Adding a fully connected Dense layer with 16 neurons and a ReLU activation
    model.add(layers.Dense(units=16, activation="relu"))

    # Adding an output layer with one neuron and a softmax activation
    # This will predict the class probability for binary classification
    model.add(layers.Dense(units=1, activation="softmax"))

    return model

In [ ]:
model_CNN = create_model()
model_historyc, resultsc = model_builder(model_CNN, alpha = 0.1, epoch = 5, batch_size = 32, adam=False)

## PLOTS

In [ ]:
def plot(train, val):
# plot the accuracies
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')

# add labels and title
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')

# add legend
    plt.legend()

# show the plot
    plt.show()


In [ ]:
plot(results['Train Accuracy'], results['Val Accuracy'])

In [ ]:
plot(results1['Train Accuracy'], results1['Val Accuracy'])

In [ ]:
plot(results2['Train Accuracy'], results2['Val Accuracy'])

In [ ]:
plot(results3['Train Accuracy'], results3['Val Accuracy'])

In [ ]:
plot(results4['Train Accuracy'], results4['Val Accuracy'])

In [ ]:
plot(resultsd['Train Accuracy'], resultsd['Val Accuracy'])

In [ ]:
plot(resultsw['Train Accuracy'], resultsw['Val Accuracy'])

In [ ]:
plot(resultsc['Train Accuracy'], resultsc['Val Accuracy'])

In [ ]:
plot(results1['Train Accuracy'], results1['Val Accuracy'])

In [ ]:
plot(results2['Train Accuracy'], results2['Val Accuracy'])

In [ ]:
plot(results3['Train Accuracy'], results3['Val Accuracy'])

In [ ]:
plot(results4['Train Accuracy'], results4['Val Accuracy'])

In [ ]:
plot(resultsw['Train Loss'], resultsw['Val Loss'])

In [ ]:
plot(resultsd['Train Loss'], resultsd['Val Loss'])

In [ ]:
plot(resultsc['Train Loss'], resultsc['Val Loss'])

In [ ]:
plot(results['Train Loss'], results['Val Loss'])

In [ ]:
!tensorboard dev upload --logdir ./logs \
--name "RTAI_ASSIGNMENT_2\All Trainings" \
--description "Logs data of all Trainings models on, \
different hyperparameters for proposed layer, \
decrease in width, decrease in depth and default cnn" \
--one_shot


In [1]:
from tensorboard.plugins.hparams import api as hp

In [3]:
from platform import python_version

In [4]:
python_version()

'3.9.13'

In [ ]:
!tensorboard dev upload --logdir ./logs \
  --name "Simple experiment with MNIST" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb" \
  --one_shot